In [19]:
from bs4 import BeautifulSoup
import requests
import base64
import json
# session = requests.session()

In [20]:
RETRY_COUNT = 1

In [21]:
class PanSearch:
    
    def __init__(self,pan):
        
        self.pan = pan
        self.url_link = "https://www.karvykra.com/UPanSearchGlobalWithPanExempt.aspx"
        self.user_Agent ="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
        
    def encode_pan_number(self,pan):
        
        # we append few characters to the original pan which is further passed for b64 encoding 
        def append_char_to_pan(pan):
            a_pan = "#"+pan[0]+"xX"+pan[1]+"M"+pan[2]+"gX"+pan[3]+"i"+pan[4]+"ta"+pan[5:7]+"v"+pan[7:9]+"yt"+pan[9]
            return a_pan
        
        txtpanNo = append_char_to_pan(pan)
        conv_bytes = txtpanNo.encode()
        b64_bytes = base64.b64encode(conv_bytes)
        encoded_pan_num = b64_bytes.decode()
        #return the encoded pan
        return encoded_pan_num
    
    def home_page(self):
        
        headers = {
            
            "User-Agent":self.user_Agent
        }
        response = session.get(self.url_link,headers = headers)
        if response.status_code == 200:
            return response
        else:
            return "No Response"
        
    def parse_with_lxml(self,home_page_response):
        #we parse the html code with lxml
        soup = BeautifulSoup(home_page_response.text,'lxml')
        return soup
    
    
    def get_payload_Var(self,soup):
        #we extract the required varibales and return them 
        __VIEWSTATE = soup.find('input',attrs={'id':'__VIEWSTATE'})["value"]
        __VIEWSTATEGENERATOR = soup.find('input',attrs={'id':'__VIEWSTATEGENERATOR'})['value']
        __EVENTVALIDATION = soup.find('input',attrs= {'id':'__EVENTVALIDATION'})['value']
        
        return __VIEWSTATE,__VIEWSTATEGENERATOR,__EVENTVALIDATION
    
    def get_captcha_image(self,soup):
        #extract the div tag from the soup
        d = soup.find('div',attrs={'style':'background-color:White;color:White;'})
        #extract the image tag with its sourcelink
        img_src = d.img['src']
        #concatinate with the domain 
        img_link = "https://www.karvykra.com/"+img_src
        #fetch the image 
        img = session.get(img_link)
        #path where the image is stored
        PATH = '/home/nagaraj/Desktop/Nag/NK/Web-Crawl/Captchas/pansearch.jpeg'
        #save the captcha in the path
        with open(PATH,'wb') as f:
            f.write(img.content)
        return PATH
    
    def get_text_from_image(self,PATH):
        import cv2
        from pytesseract import pytesseract
        from pytesseract import Output
        img = cv2.imread(PATH)
        # img=cv2.resize(img,(900,1200))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
        thresh=cv2.dilate(gray,rect_kernel,iterations=1)
        image_data = pytesseract.image_to_data(thresh, output_type=Output.DICT)
        captcha_Str = ""
        for i, word in enumerate(image_data['text']):
            if word != '':
                captcha_Str = captcha_Str+word
        return captcha_Str
        
    def fetch_data(self,VIEWSTATE,VIEWSTATEGENERATOR,EVENTVALIDATION,input_pan,captcha_text):
        
        headers = {
            
            "User-Agent":self.user_Agent,
            "Origin":"https://www.karvykra.com",
            "Referer":"https://www.karvykra.com/UPanSearchGlobalWithPanExempt.aspx",
            "Content-Type":"application/x-www-form-urlencoded"
        }
        
        payload = {
            
            "__EVENTTARGET":"",
            "__EVENTARGUMENT":"",
            "__VIEWSTATE":VIEWSTATE,
            "__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
            "__EVENTVALIDATION":EVENTVALIDATION,
            "txtpanNo":input_pan,
            "txtItnrMdtCapcha":captcha_text,
            "btnSearch":"Search Now"
        }
        
        #fetching the data with the post request
        response = session.post(self.url_link,headers=headers,data=payload)
        return response
    
    def extract_data(self,soup):
        
        #this func converts the list into the dictionary
        def Convert(a):
            it = iter(a)
            res_dct = dict(zip(it, it))
            return res_dct
        #this func extracts the tables contents 
        def get_table_data(table):
            #get all the table headings and store it in the headers var
            headers = [th.text.strip() for th in table.find_all('th')]
            #get all the tables rows
            rows = table.find_all('tr')
            data = [] #initialize an empty list 
            
            #loop through the tr's and extract the td
            for row in rows:
                cells = row.find_all('td')
                if cells:
                    items = {headers[i]: cells[i].text.strip() for i in range(len(headers))}
                    data.append(items)
            return data
        #find the table in the html using the id attribute
        table = soup.find("table",attrs={"id":"PanDetails"})
        items= []
        rows = table.find_all('tr')
        for row in rows:
            cols=row.find_all('td')
            cols=[x.text.strip('\n') for x in cols]
            items.append(cols)
        #the above generates list and we require only the 5,6,7 elements in the list
        r1 = Convert(items[5])
        r2 = Convert(items[6])
        r3 = Convert(items[7])
        r3.pop('') #pop the empty key value pair 
        dict1 = {**r1, **r2, **r3} #merge all the three dictionaries
        
        #find the table in the html using the id attribute
        table2 = soup.find('table',attrs={'id':'GridView1'})
        
        val = get_table_data(table2)
        additionalDetails = val[1]
        result = {
            "krastatus": dict1,
            "additionalDetails": [additionalDetails]
        }
        #returning the result
        return json.dumps({"result":result},indent=4)
        
        
    def get_output(self):
        
        for retry_num in range(RETRY_COUNT):
            
            #this initializes a session for each request
            self.session = requests.session()
            
            #here in this site we need to enter the pan number in the encoded form
            #so we call the 'encode_pan_number' func.
            input_pan = self.encode_pan_number(self.pan)
            
            #we call the landing page to get the initial response
            home_page_response = self.home_page()
            
            #we parse the fetched html data using the lxml
            parsed_data = self.parse_with_lxml(home_page_response)
            
            #the website requires few variables as payload so we fetch these variables
            VIEWSTATE,VIEWSTATEGENERATOR,EVENTVALIDATION = self.get_payload_Var(parsed_data)
            
            #get the captcha image from the site and store it in a particular location
            captcha_response_path = self.get_captcha_image(parsed_data)
            
            #now extract the letters in the captcha using the func 'get_text_from_image'
#             captcha_text = self.get_text_from_image(captcha_response_path)
            
            captcha_text = input("Enter the Captcha code : ")
        
            #now pass the id and captcha as params for the post request to fetch the data
            second_response = self.fetch_data(VIEWSTATE,VIEWSTATEGENERATOR,EVENTVALIDATION,input_pan,captcha_text)
            
            #now parse the response with the lxml 
            second_soup = self.parse_with_lxml(second_response)

            
            #extract the requried table contents 
            output = self.extract_data(second_soup)
            
            return output
            

In [22]:
if __name__ == '__main__':
    print(PanSearch("CAGPN9774A").get_output())

Enter the Captcha code : EP2WK
{
    "result": {
        "krastatus": {
            "Status as on :": "Mar 28, 2023 12:21:21",
            "KYC Flag : ": "",
            "PAN:": "CAGPN9774A",
            "eSIGN Flag : ": "",
            "IPV Flag :": "IPV Exempt"
        },
        "additionalDetails": [
            {
                "KRA": "CVL",
                "Date Of Upload": "30-11-2021 11:47:32",
                "Status": "02",
                "Status Description": "KYC Registered",
                "Status Date": "04-12-2021 14:31:46",
                "Date Of Modification": "",
                "Modify Status": "Not Available",
                "Modify Hold Reasons": ""
            }
        ]
    }
}
